In [4]:
def make_player_dict():
    to_return = {
        "money": 3,
        "wheat field": 1,
        "ranch": 0,
        "bakery": 1,
        "cafe": 0,
        "convenience store": 0,
        "forest": 0,
        "stadium": 0,
        "tv station": 0,
        "business center": 0,
        "cheese factory": 0,
        "furniture factory": 0,
        "mine": 0,
        "family restaurant": 0,
        "apple orchard": 0,
        "market": 0,
        "train station": 0,
        "shopping mall": 0,
        "amusement park": 0,
        "radio tower": 0
    }
    return to_return

In [8]:
def simulate_game(strategy_list):
    num_players = len(strategy_list) # determine how many players there are
    player_dicts = [make_player_dict() for player in num_players] # generate a player dictionary for each player
    player_rolling = randint(0,num_players-1) # determine who is up first by choosing a random number between 0 and num_players-1
    while True: # continue until someone wins
        roll = randint(1,6) # roll the first die
        if player_dicts[player_rolling]["train station"] > 0:
            roll += randint(1,6) # roll the second die if the player has the train station
        for ii in range(num_players): # apply this roll to every player
            player_looking_at = (player_rolling+ii)%num_players # we start by tabulating the roller's money
            strategy = strategy_list[player_looking_at] # capture the individual player's strategy
            current_player_bool = (player_looking_at == player_rolling) # is this the player who rolled
            this_players_dict = player_dicts[player_looking_at] # capture the individual player's dictionary
            
            this_players_dict = strategy(roll,this_players_dict,current_player_bool,num_players) # apply the strategy
            
            if player_dicts[player_looking_at]['money'] > 40: # test if this player has won by these not quite right standards
                return player_looking_at
            
        player_rolling = (player_rolling+1)%num_players # advance to the next player if no one has won yet